In [1]:
import sys
print(sys.version)

3.8.10 (default, Nov 22 2023, 10:22:35) 
[GCC 9.4.0]


In [2]:
import uproot
import awkward as ak
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os #for looping over files in a directory
import math
import pandas as pd
from matplotlib.ticker import ScalarFormatter
import json
from pathlib import Path
import vector

First load in a file, same file as we use for the other notebooks:

In [192]:
dataset_runnable = json.loads(Path("af_v2_2_mc_onefile.json").read_text())
#print(dataset_runnable)
file_to_open=list(dataset_runnable['Znunugamma']['files'].keys())[0]
tree_name="analysis"

In [193]:
data=None
with uproot.open(file_to_open) as f:
    #print("Keys in the ROOT file:", f.keys())
    #tree = f['analysis;1']
    #tree.show("name")

    tree_name = list(f.keys())[0]  # Get the first available tree
    tree = f[tree_name]  # Load the tree

    # Print only the variable names
    #print("\n".join(tree.keys()))
    data=tree.arrays()

Now apply some basic event-level selections, and keep the subset of events passing all selections.  There are some debugging
lines where I figure out how many events are left after different stages of selection, to compare with the eventloop example.

In [194]:
print(f"all             {len(data)}")

met_mask=(data.met_met_NOSYS<250000.)
el_mask=(ak.sum(data.el_pt_NOSYS,axis=1)==0)
mu_mask=(ak.sum(data.mu_pt_NOSYS,axis=1)==0)
vgam_mask=(data.in_vgamma_overlap_7>0)
leadjet_mask=(ak.fill_none(ak.max(data.jet_pt_NOSYS,axis=1),0)>=100000.)
btag_mask=(ak.sum(data.jet_select_btag_NOSYS,axis=1)==0)

# mindphi requirement
met=vector.zip({'pt': data.met_met_NOSYS, 'eta': np.zeros(len(data)), 'phi': data.met_phi_NOSYS, 'mass': np.zeros(len(data))})
jets = vector.zip({'pt': data.jet_pt_NOSYS, 'eta': data.jet_eta, 'phi': data.jet_phi, 'mass': data.jet_m_NOSYS})
mindphi_mask=(ak.fill_none(ak.min(abs(jets.deltaphi(met)),axis=1),0)>0.4)

presel_data=data[met_mask & el_mask & mu_mask & vgam_mask & leadjet_mask & btag_mask & mindphi_mask]
print(f"post mindphi    {ak.num(presel_data,axis=0)}")

all             508801
post mindphi    116881


The cutflow matches the eventloop numbers.

Now select events that have at least one "good" photon that passes basic quality criteria.

In [195]:
ph_preselection=((presel_data.ph_pt_NOSYS>10000) & 
                 ((abs(presel_data.ph_eta)<1.37) | ((abs(presel_data.ph_eta)>1.52) & 
                                                    (abs(presel_data.ph_eta)<2.37))) &
                 (presel_data.ph_select_or_dR02Ph_NOSYS==1) &
                 (((presel_data.ph_isEM_NOSYS)&0x45fc01)==0) &
                 (presel_data.ph_select_baseline_NOSYS==1)
                )

# keep events that have at least one photon that passes all photon preselection cuts.
# the second bit here is to avoid any events that have no photons.
ph_presel_data=presel_data[(ak.any(ph_preselection,axis=1)) & (ak.fill_none(ak.max(presel_data.ph_pt_NOSYS,axis=1),0)>10000.)]

print(len(ph_presel_data))

97268


This yield is also consistent with eventloop example.

Now I'm going to try to apply all the photon cuts.  The goal here is to:

* Find the photon that passes preselection
* Inspect that photon to see whether it's tight, isolated, truth-matched

In [196]:
# define tight and loose cuts, now on the smaller data sample that only has good events
ph_preselection=((ph_presel_data.ph_pt_NOSYS>10000) & 
                 ((abs(ph_presel_data.ph_eta)<1.37) | ((abs(ph_presel_data.ph_eta)>1.52) & 
                                                       (abs(ph_presel_data.ph_eta)<2.37))) &
                 (ph_presel_data.ph_select_or_dR02Ph_NOSYS==1) &
                 ((ph_presel_data.ph_isEM_NOSYS&0x45fc01)==0) &
                 (ph_presel_data.ph_select_baseline_NOSYS==1)
                )

# get the index of the first preselected photon (which should be the leading preselected photon
indices=ak.argmax(ph_preselection,axis=1,keepdims=True)

Now define the tight/isolated/truth cuts.  we only want to apply these cuts to the preselected photon.

In [197]:
ph_tight=(ak.firsts(ph_presel_data.ph_select_tightID_NOSYS[indices])>0)
ph_iso=(ak.firsts(ph_presel_data.ph_select_tightIso_NOSYS[indices])>0)
ph_truth=((ak.firsts(ph_presel_data.ph_truthType[indices])!=16) & 
          (ak.firsts(ph_presel_data.ph_truthType[indices])!=0))

Now check the yields.

In [198]:
print(len(ph_presel_data.ph_pt_NOSYS[indices][ ph_tight &  ph_iso &  ph_truth][:,0]))
print(len(ph_presel_data.ph_pt_NOSYS[indices][ ph_tight & ~ph_iso &  ph_truth][:,0]))
print(len(ph_presel_data.ph_pt_NOSYS[indices][~ph_tight &  ph_iso &  ph_truth][:,0]))
print(len(ph_presel_data.ph_pt_NOSYS[indices][~ph_tight & ~ph_iso &  ph_truth][:,0]))
print(len(ph_presel_data.ph_pt_NOSYS[indices][ ph_tight &  ph_iso & ~ph_truth][:,0]))
print(len(ph_presel_data.ph_pt_NOSYS[indices][ ph_tight & ~ph_iso & ~ph_truth][:,0]))
print(len(ph_presel_data.ph_pt_NOSYS[indices][~ph_tight &  ph_iso & ~ph_truth][:,0]))
print(len(ph_presel_data.ph_pt_NOSYS[indices][~ph_tight & ~ph_iso & ~ph_truth][:,0]))

65414
24384
3464
2221
163
598
166
858


This is now consistent with the eventloop yields.